# Compsci 571 Homework 2
Question 2 Variable Importance in Trees and Random Forests
Yilin Gao (yg95)
Python 3.6

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from os import system

In [2]:
train = np.genfromtxt('train.csv', delimiter=',', skip_header=1)
test = np.genfromtxt('test.csv', delimiter=',', skip_header=1)

train_X = train[:, 0: -1]
train_y = train[:, -1]

test_X = test[:, 0: -1]
test_y = test[:, -1]

print(train.shape)
print(test.shape)

(500, 6)
(100, 6)


## q2a1, decision stump based on the best split

In [93]:
def count_two_classes(y, y0 = 0, y1 = 1):
    n0 = y[y == y0].shape[0]
    n1 = y[y == y1].shape[0]
    return n0, n1

In [88]:
# split a tree node on variable X[split] and threshold = thre into 2 child nodes
# parameter X: the feature matrix, shape = [n, p]
# parameter y: the label vector, shape = [n, 1]
# parameter split: the index for the splitting variable in the feature matrix, in [0, p)
# parameter thre: the splitting threshold for the splitting variable
# parameter y0: the actual value of one type of label
# parameter y1: the actual value of the other type of label
# return X_left: the feature matrix X of the subset of data with X[split] < thre, shape = [nl, p]
# return y_left: the lable vector y of the subset of data with X[split] < thre, shape = [nl, 1]
# return X_right: the feature matrix X of the subset of data with X[split] >= thre, shape = [nr, p]
# return y_right: the lable vector y of the subset of data with X[split] >= thre, shape = [nr, 1]
def split_binary_children(X, y, split, thre):
    # left branch of the splitting node
    X_left = X[X[:, split] < thre]
    y_left = y[X[:, split] < thre]
    # right branch of root
    X_right = X[X[:, split] >= thre]
    y_right = y[X[:, split] >= thre]
    return X_left, y_left, X_right, y_right

In [4]:
# computes gini index of a node with binary labels (0, 1)
# parameter n0: number of data points of one category
# parameter n1: number of data points of the other category
# return the gini index in the node
def gini(n0, n1):
    n = n0 + n1
    return 2 * n0 * n1 / (n * n)

In [102]:
# find the split split for a binary decision stump (level = 1)
# using gini index (= 2 * p * (1-p)) as the splitting criteria
# parameter X: the feature matrix, shape = [n, p]
# parameter y: the label vector, shape = [n, 1]
# parameter best_thre: the "preset" best splitting threshold for the best split variable (in binary case 0.5)
# return best: the **index** for the best splitting variable in X, in [0, p)
def best_split(X, y, best_thre):
    assert X.shape[0] == y.shape[0]
    n = X.shape[0]
    p = X.shape[1]
    best = -1
    # root
    n_root_0, n_root_1 = count_two_classes(y)
    assert n == n_root_0 + n_root_1
    root_gini = gini(n_root_0, n_root_1) 
    # the maximal gini index is the gini index at root
    min_gini = root_gini
    for j in range(0, p): # split on variable X[j] on root
        X_left, y_left, X_right, y_right = split_binary_children(X.reshape(n, p), y.reshape(n, 1), j, best_thre)        
        # left branch of root
        n_left = y_left.shape[0]
        n_left_0, n_left_1 = count_two_classes(y_left)
        assert n_left == n_left_1 + n_left_0 
        gini_left = gini(n_left_0, n_left_1)
        # right branch of root
        n_right = X_right.shape[0]
        n_right_0, n_right_1 = count_two_classes(y_right)
        assert n_right == n_right_0 + n_right_1 
        gini_right = gini(n_right_0, n_right_1)
        # gini after split
        assert n == n_left + n_right
        gini_j = n_left / n * gini_left + n_right / n * gini_right
        if gini_j < min_gini:
            best = j
            min_gini = gini_j
    return best

In [104]:
# np.random.seed(1)
# n_idx = np.random.choice(500, 400, replace = False)
best = best_split(train_X, train_y, 0.5)
print('The best split variable index is', best)
print('The best split variable is X[' + str(best + 1) + ']')

The best split variable index is 0
The best split variable is X[1]


Relavent statistics of the decision stump are computed as following:

In [99]:
_, y_left, _, y_right = split_binary_children(train_X, train_y, best, 0.5)
n_left = y_left.shape[0]
n_right = y_right.shape[0]
n_root_0, n_root_1 = count_two_classes(train_y)
n_left_0, n_left_1 = count_two_classes(y_left)
n_right_0, n_right_1 = count_two_classes(y_right)
print('Number of points in the left child:', n_left)
print('Number of points in the left child and y = 0:', n_left_0)
print('Number of points in the left child and y = 1:', n_left_1)
print('Number of points in the right child:', n_right)
print('Number of points in the right child and y = 0:', n_right_0)
print('Number of points in the right child and y = 1:', n_right_1)
gini_root = gini(n_root_0, n_root_1)
gini_left_0 = gini(n_left_0, n_left_1)
gini_right_0 = gini(n_right_0, n_right_1)
print('Gini index before split:', gini_root)
print('Gini index in the left child:', gini_left_0)
print('Gini index in the right child:', gini_right_0)

Number of points in the left child: 243
Number of points in the left child and y = 0: 209
Number of points in the left child and y = 1: 34
Number of points in the right child: 257
Number of points in the right child and y = 0: 32
Number of points in the right child and y = 1: 225
Gini index before split: 0.499352
Gini index in the left child: 0.24068146793341125
Gini index in the right child: 0.21801995488198156


Equivalently we could use Sklearn package to compute the best decision stump:

In [8]:
# best split using the sklearn package, for the picture
dt = tree.DecisionTreeClassifier(max_depth = 1)
dt = dt.fit(train_X, train_y)
dotfile = open('tree_best_split.dot', 'w')
tree.export_graphviz(dt, out_file = dotfile)
dotfile.close()
system('dot -Tpng tree_best_split.dot -o ../hw2_answer/images/tree_best_split.png')
system('rm tree_best_split.dot')

0

## q2a1, decision stump based on the best surrogate split

In [7]:
# find the best surrogate split on the root for a given best split stump
# parameter X: the feature matrix, shape = [n, p]
# parameter best: the index for the best split variable in X, in [0, p)
# parameter best_thre: the splitting threshold for the best split variable X[best]
# parameter best_surr_thre: the "preset" splitting threshold for the best surrogate split variable (in binary case 0.5)
# return best_surr: the **index** for the best surrogate split variable in X, in [0, p)
def best_surrogate_split(X, best, best_thre, best_surr_thre):
    n = X.shape[0]
    p = X.shape[1]
    assert best >= 0 and best < p
    if p == 1: # no best surrogate split variable if only 1 variable in consideration
        return -1
    # pLbLj + pRbRj for all other variables that are not the best split variable
    best_surr = -1
    best_surr_sum = 0;
    for j in range(0, p):
        if j == best: # the best split variable
            continue
        plblj = X[np.logical_and(X[:, best] < best_thre, X[:, j] < best_surr_thre)].shape[0] / n
        prbrj = X[np.logical_and(X[:, best] >= best_thre, X[:, j] >= best_surr_thre)].shape[0] / n
        if plblj + prbrj > best_surr_sum:
            best_surr = j
            best_surr_sum = plblj + prbrj
    return best_surr

In [48]:
best_surr = best_surrogate_split(train_X, best, 0.5, 0.5)
print('The best surrogate split variable index is', best_surr)
print('The best surrogate split variable is X[' + str(best_surr + 1) + ']')

The best surrogate split variable index is 1
The best surrogate split variable is X[2]


Based on comparison, the best surrogate split for X1 is X2. The splitting threshold doesn't matter since X2 values are binary. Relavent statistics of the decision stump are computed as following:

In [97]:
_, y_left, _, y_right = split_binary_children(train_X, train_y, best_surr, 0.5)
n_left = y_left.shape[0]
n_right = y_right.shape[0]
n_root_0, n_root_1 = count_two_classes(train_y)
n_left_0, n_left_1 = count_two_classes(y_left)
n_right_0, n_right_1 = count_two_classes(y_right)
print('Number of points in the left child:', n_left)
print('Number of points in the left child and y = 0:', n_left_0)
print('Number of points in the left child and y = 1:', n_left_1)
print('Number of points in the right child:', n_right)
print('Number of points in the right child and y = 0:', n_right_0)
print('Number of points in the right child and y = 1:', n_right_1)
gini_root = gini(n_root_0, n_root_1)
gini_left_1 = gini(n_left_0, n_left_1)
gini_right_1 = gini(n_right_0, n_right_1)
print('Gini index before split:', gini_root)
print('Gini index in the left child:', gini_left_1)
print('Gini index in the right child:', gini_right_1)

Number of points in the left child: 246
Number of points in the left child and y = 0: 176
Number of points in the left child and y = 1: 70
Number of points in the right child: 254
Number of points in the right child and y = 0: 65
Number of points in the right child and y = 1: 189
Gini index before split: 0.499352
Gini index in the left child: 0.4071650472602287
Gini index in the right child: 0.38083576167152333


## q2a2, 2 variable importance measures of all variables of the tree based on the best split

In [100]:
x1_importance_2 = gini_root - 243 / 500 * gini_left_0 - 257 / 500 * gini_right_0
x2_importance_3 = gini_root - 246 / 500 * gini_left_1 - 254 / 500 * gini_right_1
print(x1_importance_2, x2_importance_3)

0.2703185497750236 0.10556222981883365


## q2a3, mean squares error of prediction on the test data of 2 trees

In [101]:
# best split
yhat_test_tree_best_split = np.ones(100)
yhat_test_tree_best_split[test_X[:, best] == 0] = 0
mse_test_tree_best_split = np.sum((yhat_test_tree_best_split - test_y) ** 2) / 100
print(mse_test_tree_best_split)
# best surrogate split
yhat_test_tree_best_sur_split = np.ones(100)
yhat_test_tree_best_sur_split[test_X[:, best_surr] == 0] = 0
mse_test_tree_best_sur_split = np.sum((yhat_test_tree_best_sur_split - test_y) ** 2) / 100
print(mse_test_tree_best_sur_split)

0.1
0.27


## q2b1 grow random forest of decision stumps

M = 1000 stumps

B = 0.8 * n bootstrap training samples

K = 1, 2, 3, 4, 5 random seleted variables

In [123]:
# parameter X: the feature matrix, shape = [n, p]
# parameter y: the label vector, shape = [n, 1]
# parameter M: number of stumps to generate in the forest
# parameter b: bootstrap resample percentage
# parameter K: number of randomly selected features in each stump
# return best_dic:
# return best_surr_dic:
# return imp_5:
# return imp_6:
def random_forest(X, y, M, b, K):
    np.random.seed(111)
    best_dic = {}
    best_surr_dic = {}
    imp_5_dic = {}
    imp_6_dic = {}
    trees = np.empty([0, 3]) # split, left predict, right predict
    assert X.shape[0] == y.shape[0]
    n = X.shape[0]
    B = int(round(b * n))
    p = X.shape[1]
    assert K <= p
    for j in range(p): # initialize dics
        best_dic[j] = 0
        best_surr_dic[j] = 0
        imp_5_dic[j] = np.empty(0)
        imp_6_dic[j] = np.empty(0)
    for m in range(M): # tree
        n_idx = np.random.choice(n, B, replace = False) # indices for bootstrap samples
        n_oobidx = list(set(range(n)) - set(n_idx))
        feature_idx = np.random.choice(p, K, replace = False) # indices for selected features
        y_sample = y[n_idx, :]
        X_sample = X[n_idx, :]
        X_sample = X_sample[:, feature_idx]
        best_idx = best_split(X_sample.reshape(B, K), y_sample.reshape(B, 1), 0.5) # the "false" best split variable index in feature_idx
        if best_idx == -1: # the best split variable doesn't exist
            print('No best split when K = {}, m = {}'.format(K, m))
            continue
        best_surr_idx = best_surrogate_split(X_sample.reshape(B, K), best_idx, 0.5, 0.5) # the "false" best surrogate variable index in feature_idx
        # update counter for best split variable and best surrogate variable       
        best = feature_idx[best_idx] # the "real" best split variable index in X
        best_dic[best] = best_dic[best] + 1
        if best_surr_idx != -1: # the best surrogate splitting variable doesn't exist (K = 1)
            best_surr = feature_idx[best_surr_idx] # the "real" best surrogate variable index in X
            best_surr_dic[best_surr] = best_surr_dic[best_surr] + 1
        # importance 5
        _, y_left, _, y_right = split_binary_children(X_sample.reshape(B, K), y_sample.reshape(B, 1), best_idx, 0.5)
        n_root = y_left.shape[0] + y_right.shape[0]
        n_root_0, n_root_1 = count_two_classes(y_sample)
        assert n_root == B
        assert n_root == n_root_0 + n_root_1
        n_left = y_left.shape[0]
        n_right = y_right.shape[0]
        n_left_0, n_left_1 = count_two_classes(y_left)
        assert n_left == n_left_0 + n_left_1
        n_right_0, n_right_1 = count_two_classes(y_right)
        assert n_right == n_right_0 + n_right_1
        gini_root = gini(n_root_0, n_root_1)
        gini_left = gini(n_left_0, n_left_1)
        gini_right = gini(n_right_0, n_right_1)
        delta_gini = gini_root - n_left / n_root * gini_left - n_right / n_root * gini_right
        imp_5_dic[best] = np.append(imp_5_dic[best], delta_gini)
        # importance 6
        if n_left_0 >= n_left / 2: # the left branch is predicted as 0
            left_predict = 0
            right_predict = 1
        else:
            left_predict = 1
            right_predict = 0
        trees = np.append(trees, np.array([best, left_predict, right_predict]).reshape(1, 3), axis = 0)
        # out-of-bag
        y_oob = y[n_oobidx, :]
        X_oob = X[n_oobidx, :]
        X_oob = X_oob[:, best]
        # if X value < 0.5, goes to left, else goes to right
        yhat_oob = np.ones(n - B)
        if n_left_0 >= n_left / 2: # the left branch is predicted as 0
            yhat_oob[X_oob < 0.5] = 0
        else: # the right branch is predicted as 0
            yhat_oob[X_oob >= 0.5] = 0
        err_oob = np.sum((yhat_oob - y_oob.flatten()) ** 2) / (n - B)
        # permute X[best]
        np.random.shuffle(X_oob)
        yhat_oob_perm = np.ones(n - B)
        if n_left_0 >= n_left / 2: # the left branch is predicted as 0
            yhat_oob_perm[X_oob < 0.5] = 0
        else: # the right branch is predicted as 0
            yhat_oob_perm[X_oob >= 0.5] = 0
        err_oob_perm = np.sum((yhat_oob_perm - y_oob.flatten()) ** 2) / (n - B)
        imp_6_dic[best] = np.append(imp_6_dic[best], err_oob_perm - err_oob)
    return best_dic, best_surr_dic, imp_5_dic, imp_6_dic, trees

## q2b1

For each K = 1, 2, 3, 4, 5:

    how many times each variable is the best split
    
    how many times each variable is the best surrogate split

In [124]:
best_dic_list = []
imp_5_list = []
imp_6_list = []
trees_list = []
for k in range(1, 6):
    best_dic, best_surr_dic, imp_5_dic, imp_6_dic, trees = random_forest(train_X, train_y.reshape((500, 1)), 1000, 0.8, k)
    best_dic_list.append(best_dic)
    imp_5_list.append(imp_5_dic)
    imp_6_list.append(imp_6_dic)
    trees_list.append(trees)
    print('K = ' + str(k) + ':') 
    print('The map for best split variable is:', best_dic)
    print('The map for best surrogate split variable is:', best_surr_dic)
    print('====================')

No best split when K = 1, m = 600
K = 1:
The map for best split variable is: {0: 215, 1: 203, 2: 186, 3: 192, 4: 203}
The map for best surrogate split variable is: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}
K = 2:
The map for best split variable is: {0: 399, 1: 312, 2: 119, 3: 116, 4: 54}
The map for best surrogate split variable is: {0: 0, 1: 106, 2: 291, 3: 268, 4: 335}
K = 3:
The map for best split variable is: {0: 585, 1: 321, 2: 48, 3: 39, 4: 7}
The map for best surrogate split variable is: {0: 0, 1: 291, 2: 288, 3: 146, 4: 275}
K = 4:
The map for best split variable is: {0: 788, 1: 212, 2: 0, 3: 0, 4: 0}
The map for best surrogate split variable is: {0: 0, 1: 582, 2: 168, 3: 22, 4: 228}
K = 5:
The map for best split variable is: {0: 1000, 1: 0, 2: 0, 3: 0, 4: 0}
The map for best surrogate split variable is: {0: 0, 1: 1000, 2: 0, 3: 0, 4: 0}


## q2b2

For each k = 1, 2, 3, 4, 5:

    compute 2 variable importance measures for each variable

In [115]:
for k in range(5):
    imp_5 = imp_5_list[k]
    imp_6 = imp_6_list[k]
    print('K = {}'.format(k + 1))
    for j in range(5): # variable
        imp = imp_5[j].flatten()
        imp = np.mean(imp)
        print('Variable importance (5) for variable {} is {}.'.format(j, imp))
        imp = imp_6[j].flatten()
        imp = np.mean(imp)
        print('Variable importance (6) for variable {} is {}.'.format(j, imp))

K = 1
Variable importance (5) for variable 0 is 0.26941944916060806.
Variable importance (6) for variable 0 is 0.37032558139534877.
Variable importance (5) for variable 1 is 0.10559694666254214.
Variable importance (6) for variable 1 is 0.22847290640394086.
Variable importance (5) for variable 2 is 0.0009261358342768216.
Variable importance (6) for variable 2 is -0.024193548387096784.
Variable importance (5) for variable 3 is 0.000907843322142195.
Variable importance (6) for variable 3 is 0.009479166666666664.
Variable importance (5) for variable 4 is 0.0004051098267352955.
Variable importance (6) for variable 4 is -0.033300492610837444.
K = 2
Variable importance (5) for variable 0 is 0.2702288467576328.
Variable importance (6) for variable 0 is 0.3647117794486215.
Variable importance (5) for variable 1 is 0.1061495545578077.
Variable importance (6) for variable 1 is 0.22564102564102567.
Variable importance (5) for variable 2 is 0.0012056344339151782.
Variable importance (6) for variab

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## q2b3

compute the mean squares loss on the test data using 2 methods:

1. use the majority vote of the stumps as the prediction

2. find the prediction of each stump, compute squares loss on each, and average the results

In [117]:
for k in range(5):
    best_dic = best_dic_list[k]
    trees = trees_list[k]
    M = trees.shape[0] # may not be 1000
    yhat_test_votes = np.zeros(100) # count of vote for 1
    err1 = 0
    err2 = 0
    for m in range(M):
        split = int(trees[m, 0])
        left_predict = trees[m, 1]
        yhat_test = np.ones(100)
        X_split = test_X[:, split]
        if left_predict == 0:
            yhat_test[X_split < 0.5] = 0
        else:
            yhat_test[X_split >= 0.5] = 0
        yhat_test_votes = yhat_test_votes + yhat_test # if the current vote is 1, add 1 to yhat_test_votes count
        err_tree = np.sum((yhat_test.flatten() - test_y) ** 2) / test_y.shape[0]
        err2 = err2 + err_tree
    yhat_test_votes[yhat_test_votes <= 500] = 0
    yhat_test_votes[yhat_test_votes > 500] = 1
    # numpy vector and 1d array are DIFFERENT!!!
    err1 = np.sum((yhat_test_votes.flatten() - test_y) ** 2) / test_y.shape[0]
    err2 = err2 / 1000
    print(err1, err2)

0.17 0.35823999999999867
0.15 0.26290999999999953
0.1 0.18998999999999874
0.1 0.13603999999999788
0.1 0.09999999999999859


## q2c grow random forest of decision stumps

B = q * n, q = 0.4, 0.5, 0.6, 0.7, 0.8

K = 2

M = 1000

In [118]:
best_dic_list = []
imp_5_list = []
imp_6_list = []
trees_list = []
for q in range(4, 9):
    best_dic, best_surr_dic, imp_5_dic, imp_6_dic, trees = random_forest(train_X, train_y.reshape((500, 1)), 1000, q / 10, 2)
    best_dic_list.append(best_dic)
    imp_5_list.append(imp_5_dic)
    imp_6_list.append(imp_6_dic)
    trees_list.append(trees)
    print('q = {}:'.format(q / 10)) 
    print('The map for best split variable is:', best_dic)
    print('The map for best surrogate split variable is:', best_surr_dic)
    print('====================')

q = 0.4:
The map for best split variable is: {0: 406, 1: 283, 2: 103, 3: 104, 4: 104}
The map for best surrogate split variable is: {0: 0, 1: 108, 2: 299, 3: 305, 4: 288}
q = 0.5:
The map for best split variable is: {0: 415, 1: 318, 2: 91, 3: 91, 4: 85}
The map for best surrogate split variable is: {0: 0, 1: 107, 2: 305, 3: 275, 4: 313}
q = 0.6:
The map for best split variable is: {0: 392, 1: 301, 2: 103, 3: 128, 4: 76}
The map for best surrogate split variable is: {0: 0, 1: 96, 2: 283, 3: 275, 4: 346}
q = 0.7:
The map for best split variable is: {0: 390, 1: 294, 2: 141, 3: 115, 4: 60}
The map for best surrogate split variable is: {0: 0, 1: 103, 2: 275, 3: 270, 4: 352}
q = 0.8:
The map for best split variable is: {0: 399, 1: 312, 2: 119, 3: 116, 4: 54}
The map for best surrogate split variable is: {0: 0, 1: 106, 2: 291, 3: 268, 4: 335}


## q2c1 variable importance in (5) and (6)

In [119]:
for q in range(5):
    imp_5 = imp_5_list[q]
    imp_6 = imp_6_list[q]
    print('q = {}'.format((q + 4) / 10))
    for j in range(5): # variable
        imp = imp_5[j].flatten()
        imp = np.mean(imp)
        print('Variable importance (5) for variable {} is {}.'.format(j, imp))
        imp = imp_6[j].flatten()
        imp = np.mean(imp)
        print('Variable importance (6) for variable {} is {}.'.format(j, imp))

q = 0.4
Variable importance (5) for variable 0 is 0.2696168961880055.
Variable importance (6) for variable 0 is 0.3669786535303777.
Variable importance (5) for variable 1 is 0.10576785698493632.
Variable importance (6) for variable 1 is 0.23422850412249704.
Variable importance (5) for variable 2 is 0.003899756845279793.
Variable importance (6) for variable 2 is -0.007961165048543696.
Variable importance (5) for variable 3 is 0.0040542737776870135.
Variable importance (6) for variable 3 is -0.0007692307692307742.
Variable importance (5) for variable 4 is 0.002572356435299024.
Variable importance (6) for variable 4 is -0.015064102564102563.
q = 0.5
Variable importance (5) for variable 0 is 0.2696419134600412.
Variable importance (6) for variable 0 is 0.36537831325301207.
Variable importance (5) for variable 1 is 0.10540532507373887.
Variable importance (6) for variable 1 is 0.22784905660377358.
Variable importance (5) for variable 2 is 0.0026316560853829815.
Variable importance (6) for v

## q2c2 standard deviation of variable imporance in (5) and (6)

In [120]:
for q in range(5):
    imp_5 = imp_5_list[q]
    imp_6 = imp_6_list[q]
    print('q = {}'.format((q + 4) / 10))
    for j in range(5): # variable
        imp = imp_5[j]
        std = np.std(imp)
        print('Standard deviation of variable importance (5) for variable {} is {}.'.format(j, std))
        imp = imp_6[j]
        std = np.std(imp)
        print('Standard deviation of variable importance (6) for variable {} is {}.'.format(j, std))

q = 0.4
Standard deviation of variable importance (5) for variable 0 is 0.02729892385445329.
Standard deviation of variable importance (6) for variable 0 is 0.03144766695159181.
Standard deviation of variable importance (5) for variable 1 is 0.022169085572470368.
Standard deviation of variable importance (6) for variable 1 is 0.034970170757314345.
Standard deviation of variable importance (5) for variable 2 is 0.003543632193578646.
Standard deviation of variable importance (6) for variable 2 is 0.03305460491498919.
Standard deviation of variable importance (5) for variable 3 is 0.003339324648230455.
Standard deviation of variable importance (6) for variable 3 is 0.0387993710001409.
Standard deviation of variable importance (5) for variable 4 is 0.002730278242917385.
Standard deviation of variable importance (6) for variable 4 is 0.03365225881482412.
q = 0.5
Standard deviation of variable importance (5) for variable 0 is 0.02140012508633232.
Standard deviation of variable importance (6)